In [3]:
!pip install sentence-transformers

In [4]:
import os
from dotenv import load_dotenv
from supabase import create_client
from sentence_transformers import SentenceTransformer

load_dotenv()

# ---------------------
# Load model
# ---------------------
model = SentenceTransformer("all-MiniLM-L6-v2")

# ---------------------
# Supabase client
# ---------------------
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


# ---------------------
# Get embedding from HF
# ---------------------
def get_embedding(text):
    return model.encode(text).tolist()


# ---------------------
# Insert sample math problems
# ---------------------
sample_problems = [
    "What is the derivative of x^2?",
    "Solve the equation 3x - 7 = 11",
    "Find the area of a triangle with base 10 and height 5"
]

def insert_samples():
    for prob in sample_problems:
        emb = get_embedding(prob)
        supabase.table("math_problems").insert({
            "problem_text": prob,
            "embedding": emb
        }).execute()
    print("Inserted sample problems!")


# ---------------------
# Search for similar problem
# ---------------------
def search_similar(query_text, limit=1):
    query_emb = get_embedding(query_text)

    result = supabase.rpc(
        "match_math_problems",
        {
            "query_embedding": query_emb,
            "match_count": limit
        }
    ).execute()

    return result.data


# ---------------------
# Main
# ---------------------
if __name__ == "__main__":
    # Run only once:
    # insert_samples()

    query = "How do you differentiate a polynomial?"
    matches = search_similar(query)

    print("\nQuery:", query)
    print("Most similar problem:")
    print(matches)



Query: How do you differentiate a polynomial?
Most similar problem:
[{'id': 3, 'problem_text': 'Solve 7 × 8.', 'distance': 0.137293266896017}]


In [5]:
from supabase import create_client
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv

load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


problems = [
    "What is 12 + 5?",
    "Solve 7 × 8.",
    "Find the perimeter of a triangle with sides 3, 4, 5.",
    "What is the square root of 144?",
    "If a train travels 60 km in 1 hour, how far in 4 hours?"
]

for text in problems:
    embedding = model.encode(text).tolist()

    data = {
        "problem_text": text,
        "embedding": embedding
    }

    response = supabase.table("math_problems").insert(data).execute()

    print(f"Inserted → {text}")

print("\nAll problems inserted successfully!")



Inserted → What is 12 + 5?
Inserted → Solve 7 × 8.
Inserted → Find the perimeter of a triangle with sides 3, 4, 5.
Inserted → What is the square root of 144?
Inserted → If a train travels 60 km in 1 hour, how far in 4 hours?

All problems inserted successfully!


In [6]:
def find_similar_problem(query:str, top_k: int=3):
    embedding = model.encode(query).tolist()

    response = supabase.rpc("match_math_problems",
    {
        "query_embedding": embedding,
        "match_count": top_k
    }).execute()

    return response.data

In [7]:
if __name__ == "__main__":
    query = "how much is 14+3?"
    results = find_similar_problem(query)

    print("\n query",query)
    print("\n top matches:")

    for r in results:
        print(f"ID :{r['id']}")
        print(f"problem: {r['problem_text']}")
        print(f"similarity score: {r['distance']:.4f}")
        print("-" * 40)


 query how much is 14+3?

 top matches:
ID :2
problem: What is 12 + 5?
similarity score: 0.5766
----------------------------------------
ID :7
problem: What is 12 + 5?
similarity score: 0.5766
----------------------------------------
ID :3
problem: Solve 7 × 8.
similarity score: 0.4135
----------------------------------------


In [1]:
import redis

r = redis.Redis(
    host='redis-17806.crce206.ap-south-1-1.ec2.cloud.redislabs.com',
    port=17806,
    decode_responses=True,
    username="default",
    password="HXCtxpx907Kcs80VAk8qHnAlaoaL0xCY",
)

success = r.set('foo', 'bar')
# True

result = r.get('foo')
print(result)


bar


In [8]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [14]:
import redis
import pickle

r = redis.Redis(
    host='redis-17806.crce206.ap-south-1-1.ec2.cloud.redislabs.com',
    port=17806,
    decode_responses=False,   # important for binary embeddings
    username="default",
    password="HXCtxpx907Kcs80VAk8qHnAlaoaL0xCY",
)


In [15]:
import hashlib
import pickle 

def get_cache_key(text: str):
    return "embedding:" + hashlib.sha256(text.encode()).hexdigest()

def get_embedding(text: str):
    key = get_cache_key(text)
    cached = r.get(key)
    
    if cached:
        print("loaded from redis cache")
        return pickle.loads(cached)
    
    print("computing new embeddings")
    embedding = model.encode(text).tolist()

    r.set(key, pickle.dumps(embedding))

    return embedding

In [16]:
text1 = "What is the derivative of x^2?"
emb1 = get_embedding(text1)

print(len(emb1))

computing new embeddings
384


In [17]:
import time 
import redis 
import pickle 

r = redis.Redis(
    host='redis-17806.crce206.ap-south-1-1.ec2.cloud.redislabs.com',
    port=17806,
    decode_responses=False,   # important for binary embeddings
    username="default",
    password="HXCtxpx907Kcs80VAk8qHnAlaoaL0xCY",
)

def slow_calculation(x, y):
    key = f"calc:{x}:{y}"

    cached_result = r.get(key)

    if cached_result:
        print("using cached reuslt")
        return pickle.loads(cached_result)
    
    print("doing calculation..............")
    time.sleep(3)
    result = x * y + x - y 

    r.set(key, pickle.dumps(result))
    return result 

print(slow_calculation(10,5))
print(slow_calculation(3,4))
print(slow_calculation(10,5))

doing calculation..............
55
doing calculation..............
11
using cached reuslt
55


In [19]:
print(r.keys("*"))

[b'embedding:e8e2ef358da1dd1380af3b9a6b0ff2a1a6ed54c969a053b48ae772a4649f5179', b'foo', b'calc:3:4', b'calc:10:5']


In [20]:
for key in r.keys("*"):
    print(key, "=>", r.get(key))

b'embedding:e8e2ef358da1dd1380af3b9a6b0ff2a1a6ed54c969a053b48ae772a4649f5179' => b'\x80\x04\x95\x85\r\x00\x00\x00\x00\x00\x00]\x94(G?\xb5\xbf\xf3\x00\x00\x00\x00G?\xaf2\t\x00\x00\x00\x00G\xbf\x9a\\> \x00\x00\x00G?\xb5S\xde\xa0\x00\x00\x00G\xbf\xa0\'\xc4\xe0\x00\x00\x00G\xbf\x96x\xb1\x00\x00\x00\x00G\xbf\x86\xed\xbd@\x00\x00\x00G?v\xbe\xdf\xa0\x00\x00\x00G?\xb1\xb8N\x80\x00\x00\x00G?\x8ftY\xe0\x00\x00\x00G?\xc0\xbe \xc0\x00\x00\x00G?\x90\x83\xfc \x00\x00\x00G\xbf\x97\x00\x0b\xa0\x00\x00\x00G?\xa4\xc5\x87\x00\x00\x00\x00G\xbf\xaa\x13\xb1 \x00\x00\x00G?\xa1\xe0\xed`\x00\x00\x00G\xbf\x90lp\xe0\x00\x00\x00G?\x9eO``\x00\x00\x00G\xbf\xa9[\xb6\xa0\x00\x00\x00G?\x9a\xd0\xa2 \x00\x00\x00G?\xaeO\x0f`\x00\x00\x00G\xbf\x98\xe3\xce\xe0\x00\x00\x00G\xbf\x97\r\xb3\x00\x00\x00\x00G\xbf\xa0\xfe\x1e@\x00\x00\x00G?\xa0\xa5*@\x00\x00\x00G\xbf\xa1\x9f4\xa0\x00\x00\x00G\xbf\xa1xo@\x00\x00\x00G\xbf\x8b\x1a\xd4\x00\x00\x00\x00G?\x8f\x83M@\x00\x00\x00G?x%L\xe0\x00\x00\x00G\xbf\xad\x08S`\x00\x00\x00G\xbf\xa8\xea